# [chibicc](https://github.com/rui314/chibicc)

## va_list, va_start, va_arg

```c
// Takes a printf-style format string and returns a formatted string.
char *format(char *fmt, ...) {
  char *buf;
  size_t buflen;
  FILE *out = open_memstream(&buf, &buflen);

  va_list ap;
  va_start(ap, fmt);
  vfprintf(out, fmt, ap);
  va_end(ap);
  fclose(out);
  return buf;
}
```

1. va_* 这几个宏原理就是移动指针，如下

```c
typedef unsigned char *va_list;
#define va_start(list, param) (list = (((va_list)&param) + sizeof(param)))
#define va_arg(list, type)    (*(type *)((list += sizeof(type)) - sizeof(type)))

```

只不过在GCC里不是明确用宏定义的，而是内建函数


![](resources/01.png)

## tokenize.c

```c
// Tokenize a given string and returns new tokens.
Token *tokenize(File *file) {
  current_file = file;

  char *p = file->contents;
  Token head = {};
  Token *cur = &head;

  at_bol = true;
  has_space = false;

  while (*p) {
    // Skip line comments.
    if (startswith(p, "//")) {
      p += 2;
      while (*p != '\n')
        p++;
      has_space = true;
      continue;
    }

    // Skip block comments.
    if (startswith(p, "/*")) {
      char *q = strstr(p + 2, "*/");
      if (!q)
        error_at(p, "unclosed block comment");
      p = q + 2;
      has_space = true;
      continue;
    }

    // Skip newline.
    if (*p == '\n') {
      p++;
      at_bol = true;
      has_space = false;
      continue;
    }

    // Skip whitespace characters.
    if (isspace(*p)) {
      p++;
      has_space = true;
      continue;
    }

    // Numeric literal
    if (isdigit(*p) || (*p == '.' && isdigit(p[1]))) {
      char *q = p++;
      for (;;) {
        if (p[0] && p[1] && strchr("eEpP", p[0]) && strchr("+-", p[1]))
          p += 2;
        else if (isalnum(*p) || *p == '.')
          p++;
        else
          break;
      }
      cur = cur->next = new_token(TK_PP_NUM, q, p);
      continue;
    }

```

把文件token化，比较简单，主要就几种情况

1.注释 2.换行 3.空白 4.数字常量 5.字符串常量 6.关键字 7.运算符

## preprocess.c

[Macro Algo: Dave Prosser Algo](resources/cpp.algo.pdf)

[GCC Macros](https://gcc.gnu.org/onlinedocs/cpp/Macros.html)

![](resources/02.png)

Dave 算法：

1. 每个token都有一个hideset，表示这个token之前是由哪个macro（string）替换来的。初始的时候都是空的（{}）

2. 在macro expand的过程中，如果当前这个token的string在hideset中，说明之前已经发生过一次替换，那么这次就不再替换（这就防止了多次替换）。
   
   
3. 如果hideset中没有出现过，就把当前token替换成对应的macro，同时将原来token的string放入到hideset中（也就是代码中的 $HS \cup \{T\}$）

4. 如果是function-like 的macro（注：macro定义时的参数较parameter，传入的参数叫actual或argument），先对传入的参数（actuals）作macro expand，然后在用expanded之后的actual去替换macro中的parameter，同时actual的hideset `HS'`和parameter的hideset `HS`取交集，也就是代码中的$(HS \cap HS')$

5. 在代码中的macro expand，针对每一个token，会循环去作expand，直到当前的token无法再expand了，再去处理下一个token。这样就解决了macro嵌套定义的问题，例如

    ```c
    #define A 1
    #define B A
    #define C B
    #define D C+B
    ```


## some functions

```c
/* Open a stream that writes into a malloc'd buffer that is expanded as
   necessary.  *BUFLOC and *SIZELOC are updated with the buffer's location
   and the number of characters written on fflush or fclose.  */
extern FILE *open_memstream (char **__bufloc, size_t *__sizeloc) __THROW
  __attribute_malloc__ __attr_dealloc_fclose __wur;
```